In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
from autoeq.constants import PEQ_CONFIGS
from autoeq.batch_processing import batch_processing
ROOT_PATH = Path().resolve()
import ipywidgets as widgets
from IPython.display import display
from measurements.rtings.rtings_crawler import RtingsCrawler
from measurements.crinacle.crinacle_crawler import CrinacleCrawler
from measurements.oratory1990.oratory1990_crawler import Oratory1990Crawler
from measurements.average import average_measurements
from measurements.rename_measurements import rename_measurements
from results.prune_results import prune_results
from results.update_indexes import update_all_indexes
from webapp.create_data import write_entries_and_measurements

## Crawling and Parsing
Additional Python packages are required for processing the measurements:
```bash
python -m pip install -U -r measurements/dev-requirements.txt
```

Measurement crawlers require [Google Chrome](https://www.google.com/chrome/) installed and
[ChromeDriver](https://googlechromelabs.github.io/chrome-for-testing/) binary in the measurements folder (or anywhere
in the PATH).

Measurement crawlers also require C++. This should be installed by default on Linux but on Windows you need to install
Microsoft Visual Studio build tools for this. https://visualstudio.microsoft.com/downloads/ ->
"Tools for Visual Studio 2019" -> "Build Tools for Visual Studio 2019".

oratory1990 crawler requires Ghostscript installed: https://www.ghostscript.com/download/gsdnld.html

### Crinacle
Download measurement data from Drive folder to `measurements/crinacle/raw_data/` before running this!

* `IEM Measurements/IEC60318-4 IEM Measurements (TSV txt)` into `AutoEq/measurements/crinacle/raw_data/IEC60318-4 IEM Measurements (TSV txt)`
* `IEM Measurements/4620 IEM Measurements` into `AutoEq/measurements/crinacle/raw_data/4620 IEM Measurements`
* `HP Measurements/EARS + 711 (TSV txt) (Legacy)` into `AutoEq/measurements/crinacle/raw_data/EARS + 711 (TSV txt) (Legacy)`
* `GRAS 43AG-7` into `AutoEq/measurements/crinacle/raw_data/GRAS 43AG-7`

In [54]:
crawler = CrinacleCrawler()
crawler.process()
display(crawler.widget)

Headphones with unknown manufacturers:
  * HD600 S3 (2020, fresh pads)
  * HD650 S2 (2020, fresh pads)
  * HD650 S2 (2020, worn pads)
Add them to manufacturers.tsv and run this cell again


#### Average Measurements
Crinacle's files have separate measurements for different units, to create one that users should be using, the different samples need to be averaged

**Note** If new samples have appeared for existing headphones that had only one sample, rename the original first with the renaming tool below!

In [ ]:
average_measurements(input_dir=ROOT_PATH.joinpath('measurements', 'crinacle', 'data', 'over-ear', 'GRAS 43AG-7'))
average_measurements(input_dir=ROOT_PATH.joinpath('measurements', 'crinacle', 'data', 'in-ear', '711'))
average_measurements(input_dir=ROOT_PATH.joinpath('measurements', 'crinacle', 'data', 'in-ear', 'Bruel & Kjaer 4620'))

### oratory1990
oratory1990 crawler fetches all measurements from https://www.reddit.com/r/oratory1990/wiki/index/list_of_presets/, downloads PDFs and reads the frequency response measurement data from the PDFs. Parsing the PDFs requires [Ghostscript](https://www.ghostscript.com/download/gsdnld.html) to be installed on the system.

In [39]:
crawler = Oratory1990Crawler()
crawler.process(new_only=True)
display(crawler.widget)

### Rtings
Rtings crawler fetches all measurements from https://www.rtings.com/headphones/1-5/graph and downloads and parses the JSON data files.

In [59]:
crawler = RtingsCrawler()
crawler.process()
display(crawler.widget)

## Rename Measurements
Sometimes measurements are named incorrectly or previously only one sample existed and now multiple samples have been measured and so the original one needs to be renamed as "<name> (sample 1)"

In [86]:
renames = [
    {'old_name': '64 Audio A18s (universal) (sample 1)', 'new_name': '64 Audio A18s (sample 1)', 'dbs': ['crinacle']},
    {'old_name': '64 Audio A18s (universal) (sample 2)', 'new_name': '64 Audio A18s (sample 2)', 'dbs': ['crinacle']},
    {'old_name': '64 Audio tia Trio', 'new_name': '64 Audio tia Trio (sample 1)', 'dbs': ['crinacle']},
    {'old_name': '64 Audio U12t', 'new_name': '64 Audio U12t (sample 1)', 'dbs': ['crinacle']},
    {'old_name': 'AAW (Advanced AcousticWerkes) A3H Plus', 'new_name': 'AAW A3H Plus', 'dbs': ['crinacle']},
    {'old_name': 'AAW A3H 2019', 'new_name': 'AAW A3H (2019)', 'dbs': ['crinacle']},
    {'old_name': 'AAW AXH 2019', 'new_name': 'AAW AXH (2019)', 'dbs': ['crinacle']},
    {'old_name': 'Acoustic Research AR H1', 'new_name': 'Acoustic Research AR-H1', 'dbs': ['innerfidelity']},
    {'old_name': 'AKG K340 Stock', 'new_name': 'AKG K340', 'dbs': ['innerfidelity']},
    {'old_name': 'AKG K391-NC', 'new_name': 'AKG K391NC', 'dbs': ['rtings']},
    {'old_name': 'AKG K490-NC', 'new_name': 'AKG K490NC', 'dbs': ['rtings']},
    {'old_name': 'AKG K601 2007', 'new_name': 'AKG K601 (2007)', 'dbs': ['innerfidelity']},
    {'old_name': 'Anker SoundCore Liberty 4 NC', 'new_name': 'Anker Soundcore Liberty 4 NC', 'dbs': ['rtings']},
    {'old_name': 'Anker SoundCore Liberty Air', 'new_name': 'Anker Soundcore Liberty Air', 'dbs': ['rtings']},
    {'old_name': 'Anker SoundCore Liberty Air 2', 'new_name': 'Anker Soundcore Liberty Air 2', 'dbs': ['crinacle', 'oratory1990', 'rtings']},
    {'old_name': 'Anker SoundCore Liberty Lite', 'new_name': 'Anker Soundcore Liberty Lite', 'dbs': ['rtings']},
    {'old_name': 'Anker SoundCore Life 2', 'new_name': 'Anker Soundcore Life 2', 'dbs': ['rtings']},
    {'old_name': 'Anker SoundCore Life P2', 'new_name': 'Anker Soundcore Life P2', 'dbs': ['rtings']},
    {'old_name': 'Anker SoundCore Life P3', 'new_name': 'Anker Soundcore Life P3', 'dbs': ['rtings']},
    {'old_name': 'Anker SoundCore Space NC', 'new_name': 'Anker Soundcore Space NC', 'dbs': ['rtings']},
    {'old_name': 'Anker SoundCore Space One', 'new_name': 'Anker Soundcore Space One', 'dbs': ['rtings']},
    {'old_name': 'Apple In-Ear 2013', 'new_name': 'Apple In-Ear (2013)', 'dbs': ['innerfidelity']},
    {'old_name': 'Astell & Kern Rosie (2 o’clock)', 'new_name': 'Astell & Kern Rosie (2 o\'clock)', 'dbs': ['crinacle']},
    {'old_name': 'Astell & Kern T8iE Mk1 (sample 1)', 'new_name': 'Astell & Kern T8iE (sample 1)', 'dbs': ['crinacle']},
    {'old_name': 'Astell & Kern T8iE Mk1 (sample 2)', 'new_name': 'Astell & Kern T8iE (sample 2)', 'dbs': ['crinacle']},
    {'old_name': 'ASTRO Gaming A10 Gen 2', 'new_name': 'ASTRO Gaming A10 (Gen 2)', 'dbs': ['rtings']},
    {'old_name': 'ASTRO Gaming A20 Gen 2', 'new_name': 'ASTRO Gaming A20 (Gen 2)', 'dbs': ['rtings']},
    {'old_name': 'ASTRO Gaming A40 2019', 'new_name': 'ASTRO Gaming A40 (2019)', 'dbs': ['rtings']},
    {'old_name': 'ASTRO Gaming A50 Gen 3 2017', 'new_name': 'ASTRO Gaming A50 (Gen 3 2017)', 'dbs': ['rtings']},
    {'old_name': 'ASTRO Gaming A50 Gen 4 2019', 'new_name': 'ASTRO Gaming A50 (Gen 4 2019)', 'dbs': ['rtings']},
    {'old_name': 'Audeze EL8 Closed', 'new_name': 'Audeze EL-8 Closed Back', 'dbs': ['innerfidelity']},
    {'old_name': 'Audeze EL8 Open', 'new_name': 'Audeze EL-8 Open Back', 'dbs': ['innerfidelity']},
    {'old_name': 'Audeze LCD-1 2009', 'new_name': 'Audeze LCD-1 (2009)', 'dbs': ['innerfidelity']},
    {'old_name': 'Audeze LCD-2 (pre 2017)', 'new_name': 'Audeze LCD-2 (pre-2017)', 'dbs': ['oratory1990']},
    {'old_name': 'Audeze LCD-2 (Rosewood)', 'new_name': 'Audeze LCD-2', 'dbs': ['crinacle']},
    {'old_name': 'Audeze LCD-2 Fazor', 'new_name': 'Audeze LCD-2 (Fazor)', 'dbs': ['oratory1990', 'innerfidelity']},
    {'old_name': 'Audeze LCD-2 Rev 2', 'new_name': 'Audeze LCD-2 (Rev 2)', 'dbs': ['innerfidelity', 'headphonecom']},
    {'old_name': 'Audeze LCD-3 Fazor', 'new_name': 'Audeze LCD-3 (Fazor)', 'dbs': ['innerfidelity']},
    {'old_name': 'Audeze LCD-3 Rev 2', 'new_name': 'Audeze LCD-3 (Rev 2)', 'dbs': ['innerfidelity']},
    {'old_name': 'Audeze LCD-X 2021', 'new_name': 'Audeze LCD-X (2021)', 'dbs': ['oratory1990']},
    {'old_name': 'Audeze LCD-X pre-2021', 'new_name': 'Audeze LCD-X (pre-2021)', 'dbs': ['oratory1990']},
    {'old_name': 'Audeze LCD-XC pre-2021', 'new_name': 'Audeze LCD-XC (pre-2021)', 'dbs': ['oratory1990']},
    {'old_name': 'Audio Zenith PMx2', 'new_name': 'Audio Zenith PMx2 (v1)', 'dbs': ['innerfidelity']},
    {'old_name': 'Audio Zenith PMx2 v1', 'new_name': 'Audio Zenith PMx2 (v1)', 'dbs': ['oratory1990']},
    {'old_name': 'Audio Zenith PMx2 v2', 'new_name': 'Audio Zenith PMx2 (v2)', 'dbs': ['oratory1990']},
    {'old_name': 'Audio-Technica ATH-ANC50iS Passive', 'new_name': 'Audio-Technica ATH-ANC50iS (passive)', 'dbs': ['innerfidelity']},
    {'old_name': 'Bang & Olufsen Beoplay H6', 'new_name': 'Bang & Olufsen Beoplay H6 (2nd Gen)', 'dbs': ['rtings']},
    {'old_name': 'Bang & Olufsen Beoplay H6 2nd Gen', 'new_name': 'Bang & Olufsen Beoplay H6 (2nd Gen)', 'dbs': ['innerfidelity']},
    {'old_name': 'Beats by Dr Dre Solo HD', 'new_name': 'Beats Solo HD', 'dbs': ['innerfidelity']},
    {'old_name': 'Beats Mixr 2014', 'new_name': 'Beats Mixr', 'dbs': ['innerfidelity']},
    {'old_name': 'Beats Solo II 2014', 'new_name': 'Beats Solo2 (2014)', 'dbs': ['innerfidelity']},
    {'old_name': 'Beats Solo3 Wired', 'new_name': 'Beats Solo3 (2016, wired)', 'dbs': ['innerfidelity']},
    {'old_name': 'Beats Solo3 Wireless', 'new_name': 'Beats Solo3 (2016, wireless)', 'dbs': ['innerfidelity']},
    {'old_name': 'Beats Solo3 (wireless, 2019).csv', 'new_name': 'Beats Solo3 Wireless (2019).csv', 'dbs': ['rtings']},
    {'old_name': 'Beats Studio 2 2014', 'new_name': 'Beats Studio 2 (2014)', 'dbs': ['innerfidelity']},
    {'old_name': 'Beats Studio 2 Bluetooth', 'new_name': 'Beats Studio 2 (Bluetooth)', 'dbs': ['innerfidelity']},
    {'old_name': 'Beats Tour 2014', 'new_name': 'Beats Tour (2014)', 'dbs': ['innerfidelity']},
    {'old_name': 'Bedphones', 'new_name': 'DubsLabs Bedphones', 'dbs': ['innerfidelity']},
    {'old_name': 'Beyerdynamic DT 100 2X2kOhm', 'new_name': 'Beyerdynamic DT 100 (2x 2k Ohm)', 'dbs': ['innerfidelity']},
    {'old_name': 'Beyerdynamic DT 150 250 Ohm', 'new_name': 'Beyerdynamic DT 150 (250 Ohm)', 'dbs': ['innerfidelity']},
    {'old_name': 'Beyerdynamic DT 250 250 Ohm', 'new_name': 'Beyerdynamic DT 250 (250 Ohm)', 'dbs': ['oratory1990', 'innerfidelity', 'headphonecom']},
    {'old_name': 'Beyerdynamic DT 250 80 Ohm', 'new_name': 'Beyerdynamic DT 250 (80 Ohm)', 'dbs': ['oratory1990', 'crinacle']},
    {'old_name': 'Beyerdynamic DT 48 E 120 Ohm', 'new_name': 'Beyerdynamic DT 48 E (120 Ohm)', 'dbs': ['innerfidelity']},
    {'old_name': 'Beyerdynamic DT 48 Loose', 'new_name': 'Beyerdynamic DT 48 (loose)', 'dbs': ['innerfidelity']},
    {'old_name': 'Beyerdynamic DT 48 S 5 Ohm', 'new_name': 'Beyerdynamic DT 48 S (5 Ohm)', 'dbs': ['innerfidelity']},
    {'old_name': 'Beyerdynamic DT 770 250 Ohm', 'new_name': 'Beyerdynamic DT 770 Pro', 'dbs': ['oratory1990']},
    {'old_name': 'Beyerdynamic DT 770 250 Ohm', 'new_name': 'Beyerdynamic DT 770 Pro (250 Ohm)', 'dbs': ['crinacle', 'rtings', 'headphonecom']},
    {'old_name': 'Beyerdynamic DT 770 250 Ohm (Dekoni hybrid earpads)', 'new_name': 'Beyerdynamic DT 770 Pro (Dekoni hybrid earpads)', 'dbs': ['oratory1990']},
    {'old_name': 'Beyerdynamic DT 770 250 Ohm (worn earpads)', 'new_name': 'Beyerdynamic DT 770 Pro (worn earpads)', 'dbs': ['oratory1990']},
    {'old_name': 'Beyerdynamic DT 770 32 Ohm', 'new_name': 'Beyerdynamic DT 770 Pro (32 Ohm)', 'dbs': ['headphonecom']},
    {'old_name': 'Beyerdynamic DT 770 600 Ohm', 'new_name': 'Beyerdynamic DT 770 Pro (600 Ohm)', 'dbs': ['innerfidelity', 'headphonecom']},
    {'old_name': 'Beyerdynamic DT 770 80 Ohm', 'new_name': 'Beyerdynamic DT 770 Pro (80 Ohm)', 'dbs': ['crinacle']},
    {'old_name': 'Beyerdynamic DT 770 M 80 Ohm', 'new_name': 'Beyerdynamic DT 770 M', 'dbs': ['oratory1990']},
    {'old_name': 'Beyerdynamic DT 880 250 Ohm', 'new_name': 'Beyerdynamic DT 880', 'dbs': ['oratory1990']},
    {'old_name': 'Beyerdynamic DT 880 250 Ohm', 'new_name': 'Beyerdynamic DT 880 (250 Ohm)', 'dbs': ['innerfidelity', 'headphonecom']},
    {'old_name': 'Beyerdynamic DT 880 250 Ohm (worn earpads)', 'new_name': 'Beyerdynamic DT 880 (worn earpads)', 'dbs': ['oratory1990']},
    {'old_name': 'Beyerdynamic DT 880 32 Ohm', 'new_name': 'Beyerdynamic DT 880', 'dbs': ['rtings']},
    {'old_name': 'Beyerdynamic DT 880 32 Ohm', 'new_name': 'Beyerdynamic DT 880 (32 Ohm)', 'dbs': ['crinacle', 'innerfidelity', 'headphonecom']},
    {'old_name': 'Beyerdynamic DT 880 600 Ohm', 'new_name': 'Beyerdynamic DT 880 (600 Ohm)', 'dbs': ['innerfidelity', 'headphonecom']},
    {'old_name': 'Beyerdynamic DT 990 250 Ohm', 'new_name': 'Beyerdynamic DT 990 Pro', 'dbs': ['rtings', 'headphonecom']},
    {'old_name': 'Beyerdynamic DT 990 250 Ohm', 'new_name': 'Beyerdynamic DT 990 Pro (250 Ohm)', 'dbs': ['crinacle']},
    {'old_name': 'Beyerdynamic DT 990 600 Ohm', 'new_name': 'Beyerdynamic DT 990 (600 Ohm)', 'dbs': ['headphonecom']},
    {'old_name': 'Beyerdynamic DT1990 (Dekoni lite Velours Earpads)', 'new_name': 'Beyerdynamic DT1990 (Dekoni Elite velour earpads)', 'dbs': ['oratory1990']},
    {'old_name': 'Beyerdynamic DT880 Black Edition', 'new_name': 'Beyerdynamic DT 880 Black Edition', 'dbs': ['oratory1990']},
    {'old_name': 'Beyerdynamic T1 2nd Generation', 'new_name': 'Beyerdynamic T1 (2nd Gen)', 'dbs': ['rtings', 'oratory1990']},
    {'old_name': 'Beyerdynamic T1 3rd Generation', 'new_name': 'Beyerdynamic T1 (3rd Gen)', 'dbs': ['oratory1990', 'crinacle']},
    {'old_name': 'Beyerdynamic T5 3rd Generation', 'new_name': 'Beyerdynamic T5 (3rd Gen)', 'dbs': ['oratory1990', 'crinacle']},
    {'old_name': 'Beyerdynamic T70 250 Ohm', 'new_name': 'Beyerdynamic T70', 'dbs': ['headphonecom']},
    {'old_name': 'Beyerdynamic T70 250 Ohm (sample 1)', 'new_name': 'Beyerdynamic T70 (sample 1) ', 'dbs': ['innerfidelity']},
    {'old_name': 'Beyerdynamic T70 250 Ohm (serial number 01111)', 'new_name': 'Beyerdynamic T70 (serial number 01111)', 'dbs': ['innerfidelity']},
    {'old_name': 'Beyerdynamic T90 250 Ohm', 'new_name': 'Beyerdynamic T90', 'dbs': ['headphonecom']},
    {'old_name': 'Campfire Audio Io', 'new_name': 'Campfire Audio IO', 'dbs': ['oratory1990']},
    {'old_name': 'Campfire Audio Vega (Foam Eartips)', 'new_name': 'Campfire Audio Vega (foam eartips)', 'dbs': ['oratory1990']},
    {'old_name': 'Dan Clark Audio Alpha Dog 2014', 'new_name': 'Dan Clark Audio Alpha Dog (2014)', 'dbs': ['innerfidelity']},
    {'old_name': 'Dan Clark Audio Mad Dog 2014', 'new_name': 'Dan Clark Audio Mad Dog (2014)', 'dbs': ['innerfidelity']},
    {'old_name': 'Denon AH-D2000 B2012', 'new_name': 'Denon AH-D2000 (2012)', 'dbs': ['innerfidelity']},
    {'old_name': 'Denon AH-D7000 B2012', 'new_name': 'Denon AH-D7000 (2012)', 'dbs': ['innerfidelity']},
    {'old_name': 'E-Mu Teak 2016', 'new_name': 'E-Mu Teak (2016)', 'dbs': ['innerfidelity']},
    {'old_name': 'E-Mu Teak Mahogany Cups', 'new_name': 'E-Mu Teak (mahogany)', 'dbs': ['innerfidelity']},
    {'old_name': 'E-Mu Teak Rosewood Cups', 'new_name': 'E-Mu Teak (rosewood)', 'dbs': ['innerfidelity']},
    {'old_name': 'EarFun Air Pro 2 2021', 'new_name': 'EarFun Air Pro 2 (2021)', 'dbs': ['crinacle']},
    {'old_name': 'Elysian Annihilator DIO 2021', 'new_name': 'Elysian Annihilator (2021)', 'dbs': ['crinacle']},
    {'old_name': 'Elysian Terminator', 'new_name': 'Elysian Terminator (sample 1)', 'dbs': ['crinacle']},
    {'old_name': 'Fatfreq Hayabusa (bass switch on)', 'new_name': 'Fatfreq Hayabusa (bass)', 'dbs': ['crinacle']},
    {'old_name': 'FAudio Mezzo (000 setting)', 'new_name': 'FAudio Mezzo (off-off-off)', 'dbs': ['crinacle']},
    {'old_name': 'FAudio Mezzo (003 setting)', 'new_name': 'FAudio Mezzo (off-off-on)', 'dbs': ['crinacle']},
    {'old_name': 'FAudio Mezzo (020 setting)', 'new_name': 'FAudio Mezzo (off-on-off)', 'dbs': ['crinacle']},
    {'old_name': 'FAudio Mezzo (023 setting)', 'new_name': 'FAudio Mezzo (off-on-on)', 'dbs': ['crinacle']},
    {'old_name': 'FAudio Mezzo (100 setting)', 'new_name': 'FAudio Mezzo (on-off-off)', 'dbs': ['crinacle']},
    {'old_name': 'FAudio Mezzo (120 setting)', 'new_name': 'FAudio Mezzo (on-on-off)', 'dbs': ['crinacle']},
    {'old_name': 'FAudio Mezzo (123 setting)', 'new_name': 'FAudio Mezzo (on-on-on)', 'dbs': ['crinacle']},
    {'old_name': 'FAudio Minor', 'new_name': 'FAudio Minor (sample 1)', 'dbs': ['crinacle']},
    {'old_name': 'FiiO FH5s (all switches on)', 'new_name': 'FiiO FH5s (on-on-on)', 'dbs': ['crinacle']},
    {'old_name': 'FiiO FH5s (bass switch on)', 'new_name': 'FiiO FH5s (on-off-off)', 'dbs': ['crinacle']},
    {'old_name': 'FiiO FH5s (bass)', 'new_name': 'FiiO FH5s (on-off-off)', 'dbs': ['crinacle']},
    {'old_name': 'FiiO FH5s (mid switch on)', 'new_name': 'FiiO FH5s (off-on-off)', 'dbs': ['crinacle']},
    {'old_name': 'FiiO FH5s (mids)', 'new_name': 'FiiO FH5s (off-on-off)', 'dbs': ['crinacle']},
    {'old_name': 'FiiO FH5s (treble switch on)', 'new_name': 'FiiO FH5s (off-off-on)', 'dbs': ['crinacle']},
    {'old_name': 'FiiO FH5s (treble)', 'new_name': 'FiiO FH5s (off-off-on)', 'dbs': ['crinacle']},
    {'old_name': 'Final Audio III', 'new_name': 'Final Audio Heaven III', 'dbs': ['crinacle']},
    {'old_name': 'Final Audio UX3000 (B, ANC on)', 'new_name': 'Final Audio UX3000 (Bluetooth, ANC on)', 'dbs': ['oratory1990']},
    {'old_name': 'FLC Technology FLC8 Bk Gy Bk', 'new_name': 'FLC Technology FLC8 (pop-rap, black LF, grey ULF, black HF)', 'dbs': ['innerfidelity']},
    {'old_name': 'FLC Technology FLC8 C C Bk Ligh', 'new_name': 'FLC Technology FLC8 (light, grey LF, grey ULF, blue HF)', 'dbs': ['innerfidelity']},
    {'old_name': 'FLC Technology FLC8 C C Gn Strin', 'new_name': 'FLC Technology FLC8 (strings-piano, clear LF, clear LF, green HF)', 'dbs': ['innerfidelity']},
    {'old_name': 'FLC Technology FLC8 CCY Voca', 'new_name': 'FLC Technology FLC8 (vocals, clear LF, clear ULF, yellow HF)', 'dbs': ['innerfidelity']},
    {'old_name': 'FLC Technology FLC8 G G Bk De', 'new_name': 'FLC Technology FLC8 (gray LF, grey ULF, black HF)', 'dbs': ['innerfidelity']},
    {'old_name': 'FLC Technology FLC8 GGBl', 'new_name': 'FLC Technology FLC8 (gray LF, grey ULF, blue HF)', 'dbs': ['innerfidelity']},
    {'old_name': 'Fostex TH-X00 Ebony', 'new_name': 'Fostex TH-X00 (ebony)', 'dbs': ['oratory1990']},
    {'old_name': 'Fostex TH-X00 Mahogany', 'new_name': 'Fostex TH-X00 (mahogany)', 'dbs': ['oratory1990']},
    {'old_name': 'Fostex TH-X00 Mahogany (sample 1)', 'new_name': 'Fostex TH-X00 (mahogany) (sample 1)', 'dbs': ['innerfidelity']},
    {'old_name': 'Fostex TH-X00 Mahogany (serial number 1927)', 'new_name': 'Fostex TH-X00 (mahogany) (serial number 1927)', 'dbs': ['innerfidelity']},
    {'old_name': 'Fostex TH-X00 Purpleheart', 'new_name': 'Fostex TH-X00 (purpleheart)', 'dbs': ['oratory1990']},
    {'old_name': 'Gold Planar GL2000 (leather earpads)', 'new_name': 'Gold Planar GL2000 (single-sided, leather earpads)', 'dbs': ['crinacle']},
    {'old_name': 'Gold Planar GL2000 (perforated hybrid earpads)', 'new_name': 'Gold Planar GL2000 (single-sided, perforated hybrid earpads)', 'dbs': ['crinacle']},
    {'old_name': 'Gold Planar GL2000 double-sided (leather earpads)', 'new_name': 'Gold Planar GL2000 (double-sided, leather earpads)', 'dbs': ['crinacle']},
    {'old_name': 'Gold Planar GL2000 double-sided (perforated hybrid earpads)', 'new_name': 'Gold Planar GL2000 (double-sided, perforated hybrid earpads)', 'dbs': ['crinacle']},
    {'old_name': 'Google Pixel Buds 2017', 'new_name': 'Google Pixel Buds (2017)', 'dbs': ['rtings']},
    {'old_name': 'Google Pixel Buds 2020', 'new_name': 'Google Pixel Buds (2020)', 'dbs': ['rtings']},
    {'old_name': 'HIFIMAN Arya Stealth', 'new_name': 'HIFIMAN Arya Stealth Magnet Version', 'dbs': ['oratory1990', 'crinacle', 'rtings']},
    {'old_name': 'HIFIMAN Deva (passive)', 'new_name': 'HIFIMAN Deva (wired)', 'dbs': ['crinacle']},
    {'old_name': 'HIFIMAN HE400i 2016', 'new_name': 'HIFIMAN HE400i (2016)', 'dbs': ['oratory1990', 'crinacle']},
    {'old_name': 'HIFIMAN HE400i 2020', 'new_name': 'HIFIMAN HE400i (2020)', 'dbs': ['oratory1990', 'crinacle']},
    {'old_name': 'HIFIMAN HE560 V1', 'new_name': 'HIFIMAN HE560 V1 (sample 1)', 'dbs': ['crinacle']},
    {'old_name': 'HIFIMAN Sundara', 'new_name': 'HIFIMAN Sundara (2018)', 'dbs': ['innerfidelity']},
    {'old_name': 'HIFIMAN Sundara 2018', 'new_name': 'HIFIMAN Sundara (pre-2020 earpads)', 'dbs': ['rtings']},
    {'old_name': 'HIFIMAN Sundara 2020', 'new_name': 'HIFIMAN Sundara (post-2020 earpads)', 'dbs': ['rtings']},
    {'old_name': 'HIFIMAN Susvara', 'new_name': 'HIFIMAN Susvara (sample 1)', 'dbs': ['crinacle']},
    {'old_name': 'JH Audio 13v2 (2 o’clock)', 'new_name': 'JH Audio 13v2 (2 o\'clock)', 'dbs': ['crinacle']},
    {'old_name': 'JH Audio 16v2 (2 o’clock)', 'new_name': 'JH Audio 16v2 (2 o\'clock)', 'dbs': ['crinacle']},
    {'old_name': 'JH Audio Angie (2 o’clock)', 'new_name': 'JH Audio Angie (2 o\'clock)', 'dbs': ['crinacle']},
    {'old_name': 'JH Audio Layla (2 o’clock)', 'new_name': 'JH Audio Layla (2 o\'clock)', 'dbs': ['crinacle']},
    {'old_name': 'JH Audio Layla AION (2 o’clock)', 'new_name': 'JH Audio Layla AION (2 o\'clock)', 'dbs': ['crinacle']},
    {'old_name': 'JH Audio Lola (2 o’clock)', 'new_name': 'JH Audio Lola (2 o\'clock)', 'dbs': ['crinacle']},
    {'old_name': 'JH Audio Roxanne (2 o’clock)', 'new_name': 'JH Audio Roxanne (2 o\'clock)', 'dbs': ['crinacle']},
    {'old_name': 'JQ Hua Jiang (sample 2)', 'new_name': 'JQ Hua Jiang', 'dbs': ['crinacle']},
    {'old_name': 'Klipsch Reference On Ear', 'new_name': 'Klipsch Reference On-Ear', 'dbs': ['crinacle']},
    {'old_name': 'Kumitate Lab KL-Focus Setting 2', 'new_name': 'Kumitate Lab KL-Focus (setting 2)', 'dbs': ['crinacle']},
    {'old_name': 'Kumitate Lab KL-Focus Setting 3', 'new_name': 'Kumitate Lab KL-Focus (setting 3)', 'dbs': ['crinacle']},
    {'old_name': 'Massdrop Nobel X', 'new_name': 'Noble Audio X', 'dbs': ['crinacle']},
    {'old_name': 'Mofasest M14 (00 setting)', 'new_name': 'Mofasest M14 (off-off)', 'dbs': ['crinacle']},
    {'old_name': 'Mofasest M14 (02 setting)', 'new_name': 'Mofasest M14 (off-on)', 'dbs': ['crinacle']},
    {'old_name': 'Mofasest M14 (10 setting)', 'new_name': 'Mofasest M14 (on-off)', 'dbs': ['crinacle']},
    {'old_name': 'Mofasest M14 (12 setting)', 'new_name': 'Mofasest M14 (on-on)', 'dbs': ['crinacle']},
    {'old_name': 'Moondrop Aria 2021', 'new_name': 'Moondrop Aria (2021)', 'dbs': ['crinacle']},
    {'old_name': 'Moondrop Aria 2021', 'new_name': 'Moondrop Aria', 'dbs': ['rtings']},
    {'old_name': 'Philips Fidelio M1 Mk1 2012', 'new_name': 'Philips Fidelio M1', 'dbs': ['innerfidelity']},
    {'old_name': 'Philips Fidelio S1 Early 2013', 'new_name': 'Philips Fidelio S1 (early 2013)', 'dbs': ['crinacle']},
    {'old_name': 'Philips Fidelio S2 2013', 'new_name': 'Philips Fidelio S2 (2013)', 'dbs': ['innerfidelity']},
    {'old_name': 'Philips Fidelio S2 Early 2013', 'new_name': 'Philips Fidelio S2 (early 2013)', 'dbs': ['innerfidelity']},
    {'old_name': 'qdc Anole V14 (0000 setting)', 'new_name': 'qdc Anole V14 (off-off-off-off)', 'dbs': ['crinacle']},
    {'old_name': 'qdc Anole V14 (0004 setting)', 'new_name': 'qdc Anole V14 (off-off-off-on)', 'dbs': ['crinacle']},
    {'old_name': 'qdc Anole V14 (0030 setting)', 'new_name': 'qdc Anole V14 (off-off-on-off)', 'dbs': ['crinacle']},
    {'old_name': 'qdc Anole V14 (0200 setting)', 'new_name': 'qdc Anole V14 (off-on-off-off)', 'dbs': ['crinacle']},
    {'old_name': 'qdc Anole V14 (1000 setting)', 'new_name': 'qdc Anole V14 (on-off-off-off)', 'dbs': ['crinacle']},
    {'old_name': 'qdc Anole V14 (1234 setting)', 'new_name': 'qdc Anole V14 (on-on-on-on)', 'dbs': ['crinacle']},
    {'old_name': 'qdc Anole VX All', 'new_name': 'qdc Anole VX (all)', 'dbs': ['crinacle']},
    {'old_name': 'qdc Dmagic (sample 2)', 'new_name': 'qdc Dmagic', 'dbs': ['crinacle']},
    {'old_name': 'QKZ VK4 Revised (sample 1)', 'new_name': 'QKZ VK4 (revised) (sample 1)', 'dbs': ['crinacle']},
    {'old_name': 'QKZ VK4 Revised (sample 2)', 'new_name': 'QKZ VK4 (revised) (sample 2)', 'dbs': ['crinacle']},
    {'old_name': 'Razer Hammerhead True Wireless 2019', 'new_name': 'Razer Hammerhead True Wireless (2019)', 'dbs': ['rtings']},
    {'old_name': 'Razer Hammerhead True Wireless 2021', 'new_name': 'Razer Hammerhead True Wireless (2021)', 'dbs': ['rtings']},
    {'old_name': 'Razer Man O\' War', 'new_name': 'Razer ManO\'War', 'dbs': ['rtings']},
    {'old_name': 'Razer Opus 2020', 'new_name': 'Razer Opus (2020)', 'dbs': ['crinacle']},
    {'old_name': 'Razer Opus 2021', 'new_name': 'Razer Opus (2021)', 'dbs': ['crinacle']},
    {'old_name': 'Samsung EO-IG955 (AKG)', 'new_name': 'Samsung AKG EO-IG955', 'dbs': ['crinacle', 'oratory1990']},
    {'old_name': 'Samsung Galaxy Buds2 Pro (bass boost EQ) (ANC mode)', 'new_name': 'Samsung Galaxy Buds2 Pro (bass boost EQ, ANC mode)', 'dbs': ['crinacle']},
    {'old_name': 'Samsung Galaxy Buds2 Pro (blear EQ) (ANC mode)', 'new_name': 'Samsung Galaxy Buds2 Pro (blear EQ, ANC mode)', 'dbs': ['crinacle']},
    {'old_name': 'Samsung Galaxy Buds2 Pro (dynamic EQ) (ANC mode)', 'new_name': 'Samsung Galaxy Buds2 Pro (dynamic EQ, ANC mode)', 'dbs': ['crinacle']},
    {'old_name': 'Samsung Galaxy Buds2 Pro (Passive mode)', 'new_name': 'Samsung Galaxy Buds2 Pro (passive mode)', 'dbs': ['crinacle']},
    {'old_name': 'Samsung Galaxy Buds2 Pro (soft EQ) (ANC mode)', 'new_name': 'Samsung Galaxy Buds2 Pro (soft EQ, ANC mode)', 'dbs': ['crinacle']},
    {'old_name': 'Samsung Galaxy Buds2 Pro (treble boost EQ) (ANC mode)', 'new_name': 'Samsung Galaxy Buds2 Pro (treble boost EQ, ANC mode)', 'dbs': ['crinacle']},
    {'old_name': 'Samsung Galaxy Buds2 Pro (treble boost EQ) (ANC mode) (sample 1)', 'new_name': 'Samsung Galaxy Buds2 Pro (treble boost EQ, ANC mode) (sample 1)', 'dbs': ['crinacle']},
    {'old_name': 'Sennheiser HD 25-1 II B (2012 model)', 'new_name': 'Sennheiser HD 25-1 II B (2012)', 'dbs': ['innerfidelity']},
    {'old_name': 'Sennheiser IE 40 PRO', 'new_name': 'Sennheiser IE 40 Pro', 'dbs': ['rtings']},
    {'old_name': 'Sennheiser MM 550 Travel Bluetooth NC', 'new_name': 'Sennheiser MM 550', 'dbs': ['headphonecom']},
    {'old_name': 'Sennheiser Momentum 4 Wireless (active)', 'new_name': 'Sennheiser Momentum 4 Wireless', 'dbs': ['crinacle']},
    {'old_name': 'Shanling ME800 (00 setting)', 'new_name': 'Shanling ME800 (off-off)', 'dbs': ['crinacle']},
    {'old_name': 'Shanling ME800 (02 setting)', 'new_name': 'Shanling ME800 (off-on)', 'dbs': ['crinacle']},
    {'old_name': 'Shanling ME800 (10 setting)', 'new_name': 'Shanling ME800 (on-off)', 'dbs': ['crinacle']},
    {'old_name': 'Shanling ME800 (12 setting)', 'new_name': 'Shanling ME800 (on-on)', 'dbs': ['crinacle']},
    {'old_name': 'Shozy x AAW Pola', 'new_name': 'Shozy x AAW Pola (sample 2)', 'dbs': ['crinacle']},
    {'old_name': 'Sony WF-1000XM3 NC', 'new_name': 'Sony WF-1000XM3 (ANC on)', 'dbs': ['crinacle']},
    {'old_name': 'Stax SR-001 Mk1 CES (stock)', 'new_name': 'Stax SR-001 Mk1 CES (CES-A1)', 'dbs': ['crinacle']},
    {'old_name': 'Stax SR-003MK2 (CES-A1 closedback adapter)', 'new_name': 'Stax SR-003MK2 (CES-A1)', 'dbs': ['oratory1990']},
    {'old_name': 'Steven Slate Audio VSX (passive plugin inactive)))', 'new_name': 'Steven Slate Audio VSX (passive, plugin inactive)', 'dbs': ['oratory1990']},
    {'old_name': 'ThieAudio Legacy 3 (00 setting)', 'new_name': 'ThieAudio Legacy 3 (off-off)', 'dbs': ['crinacle']},
    {'old_name': 'ThieAudio Legacy 3 (02 setting)', 'new_name': 'ThieAudio Legacy 3 (off-on)', 'dbs': ['crinacle']},
    {'old_name': 'ThieAudio Legacy 3 (10 setting)', 'new_name': 'ThieAudio Legacy 3 (on-off)', 'dbs': ['crinacle']},
    {'old_name': 'ThieAudio Legacy 3 (12 setting)', 'new_name': 'ThieAudio Legacy 3 (on-on)', 'dbs': ['crinacle']},
    {'old_name': 'ThieAudio Legacy 3 (CIEM, 00 setting)', 'new_name': 'ThieAudio Legacy 3 (CIEM, off-off)', 'dbs': ['crinacle']},
    {'old_name': 'ThieAudio Legacy 3 (CIEM, 02 setting)', 'new_name': 'ThieAudio Legacy 3 (CIEM, off-on)', 'dbs': ['crinacle']},
    {'old_name': 'ThieAudio Legacy 3 (CIEM, 10 setting)', 'new_name': 'ThieAudio Legacy 3 (CIEM, on-off)', 'dbs': ['crinacle']},
    {'old_name': 'ThieAudio Legacy 3 (CIEM, 12 setting)', 'new_name': 'ThieAudio Legacy 3 (CIEM, on-on)', 'dbs': ['crinacle']},
    {'old_name': 'ThieAudio Legacy 3 (universal, 00 setting)', 'new_name': 'ThieAudio Legacy 3 (off-off)', 'dbs': ['crinacle']},
    {'old_name': 'ThieAudio Legacy 3 (universal, 02 setting)', 'new_name': 'ThieAudio Legacy 3 (off-on)', 'dbs': ['crinacle']},
    {'old_name': 'ThieAudio Legacy 3 (universal, 10 setting)', 'new_name': 'ThieAudio Legacy 3 (on-off)', 'dbs': ['crinacle']},
    {'old_name': 'ThieAudio Legacy 3 (universal, 12 setting)', 'new_name': 'ThieAudio Legacy 3 (on-on)', 'dbs': ['crinacle']},
    {'old_name': 'ThieAudio Voyager 14 (00 setting)', 'new_name': 'ThieAudio Voyager 14 (off-off)', 'dbs': ['crinacle']},
    {'old_name': 'ThieAudio Voyager 14 (02 setting)', 'new_name': 'ThieAudio Voyager 14 (off-on)', 'dbs': ['crinacle']},
    {'old_name': 'ThieAudio Voyager 14 (10 setting)', 'new_name': 'ThieAudio Voyager 14 (on-off)', 'dbs': ['crinacle']},
    {'old_name': 'ThieAudio Voyager 14 (12 setting)', 'new_name': 'ThieAudio Voyager 14 (on-on)', 'dbs': ['crinacle']},
    {'old_name': 'TRI Starsea (00 setting)', 'new_name': 'TRI Starsea (off-off)', 'dbs': ['crinacle']},
    {'old_name': 'TRI Starsea (02 setting)', 'new_name': 'TRI Starsea (off-on)', 'dbs': ['crinacle']},
    {'old_name': 'TRI Starsea (10 setting)', 'new_name': 'TRI Starsea (on-off)', 'dbs': ['crinacle']},
    {'old_name': 'TRI Starsea (12 setting)', 'new_name': 'TRI Starsea (on-on)', 'dbs': ['crinacle']},
    {'old_name': 'Unique Melody MEST (universal)', 'new_name': 'Unique Melody MEST', 'dbs': ['crinacle']},
    {'old_name': 'Yanyin Canon (000 setting)', 'new_name': 'Yanyin Canon (off-off-off)', 'dbs': ['crinacle']},
    {'old_name': 'Yanyin Canon (003 setting)', 'new_name': 'Yanyin Canon (off-off-on)', 'dbs': ['crinacle']},
    {'old_name': 'Yanyin Canon (020 setting)', 'new_name': 'Yanyin Canon (off-on-off)', 'dbs': ['crinacle']},
    {'old_name': 'Yanyin Canon (100 setting)', 'new_name': 'Yanyin Canon (on-off-off)', 'dbs': ['crinacle']},
    {'old_name': 'Yanyin Canon (123 setting)', 'new_name': 'Yanyin Canon (on-on-on)', 'dbs': ['crinacle']},
]
rename_measurements(renames, dry_run=True)

Renamed "crinacle\data\in-ear\711\64 Audio A18s (universal) (sample 1).csv" as "crinacle\data\in-ear\711\64 Audio A18s (sample 1).csv"
Renamed "crinacle\data\in-ear\711\64 Audio A18s (universal) (sample 2).csv" as "crinacle\data\in-ear\711\64 Audio A18s (sample 2).csv"
Renamed "crinacle\data\in-ear\711\64 Audio tia Trio.csv" as "crinacle\data\in-ear\711\64 Audio tia Trio (sample 1).csv"
Renamed "crinacle\data\in-ear\711\64 Audio U12t.csv" as "crinacle\data\in-ear\711\64 Audio U12t (sample 1).csv"
Renamed "crinacle\data\in-ear\711\AAW (Advanced AcousticWerkes) A3H Plus.csv" as "crinacle\data\in-ear\711\AAW A3H Plus.csv"
Renamed "crinacle\data\in-ear\711\AAW A3H 2019.csv" as "crinacle\data\in-ear\711\AAW A3H (2019).csv"
Renamed "crinacle\data\in-ear\711\AAW AXH 2019.csv" as "crinacle\data\in-ear\711\AAW AXH (2019).csv"
Renamed "innerfidelity\data\over-ear\Acoustic Research AR H1.csv" as "innerfidelity\data\over-ear\Acoustic Research AR-H1.csv"
Renamed "innerfidelity\data\over-ear\AKG K34

## Prune Results
Check if obsolete results (e.g. because of renaming) exist and remove them

In [74]:
prune_results(databases=['crinacle', 'oratory1990', 'rtings', 'innerfidelity', 'headphonecom'], dry_run=False)

Removed "C:\Users\jaakko\code\AutoEq\results\innerfidelity\over-ear\AKG K240 MKII\AKG K240 MKII.png"
Removed "C:\Users\jaakko\code\AutoEq\results\innerfidelity\over-ear\AKG K601\AKG K601.png"


## Update Results
Creates new results from the measurements. `eq_kwargs` are parameters shared by all jobs.

In [5]:
eq_kwargs = {
    'parametric_eq': True, 'ten_band_eq': True, 'convolution_eq': True,
    'parametric_eq_config': [PEQ_CONFIGS['4_PEAKING_WITH_LOW_SHELF'], PEQ_CONFIGS['4_PEAKING_WITH_HIGH_SHELF']],
    'fs': [44100, 48000],
    'thread_count': 0,
}

#### oratory1990 Over-ear

In [48]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'oratory1990', 'data', 'over-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'oratory1990', 'over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### oratory1990 In-ear

In [47]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'oratory1990', 'data', 'in-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'oratory1990', 'in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'autoeq_in-ear.csv'),
    bass_boost_gain=9.5, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### oratory1990 Earbud

In [46]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'oratory1990', 'data', 'earbud'),
    output_dir=ROOT_PATH.joinpath('results', 'oratory1990', 'earbud'),
    compensation=ROOT_PATH.joinpath('compensation', 'autoeq_in-ear.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### crinacle GRAS 43AG-7 On-ear

In [45]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'crinacle', 'data', 'over-ear', 'GRAS 43AG-7'),
    output_dir=ROOT_PATH.joinpath('results', 'crinacle', 'GRAS 43AG-7 over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### crinacle EARS+711 Over-ear

In [44]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'crinacle', 'data', 'over-ear', 'EARS + 711'),
    output_dir=ROOT_PATH.joinpath('results', 'crinacle', 'EARS + 711 over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'crinacle_harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### crinacle 4620 In-ear

In [43]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'crinacle', 'data', 'in-ear', 'Bruel & Kjaer 4620'),
    output_dir=ROOT_PATH.joinpath('results', 'crinacle', 'Bruel & Kjaer 4620 in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'diffuse_field_5128_-1dBpoct.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/2 [00:00<?, ?it/s]

#### crinacle 711 In-ear

In [33]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'crinacle', 'data', 'in-ear', '711'),
    output_dir=ROOT_PATH.joinpath('results', 'crinacle', '711 in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'autoeq_in-ear.csv'),
    bass_boost_gain=9.5, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Rtings Over-ear

In [42]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'rtings', 'data', 'over-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Rtings', 'over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'rtings_harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Rtings In-ear

In [32]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'rtings', 'data', 'in-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Rtings', 'in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'rtings_autoeq_in-ear.csv'),
    bass_boost_gain=9.5, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Rtings Earbud

In [41]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'rtings', 'data', 'earbud'),
    output_dir=ROOT_PATH.joinpath('results', 'Rtings', 'earbud'),
    compensation=ROOT_PATH.joinpath('compensation', 'rtings_autoeq_in-ear.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Innerfidelity In-ear

In [40]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'innerfidelity', 'data', 'over-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Innerfidelity', 'over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'innerfidelity_harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Innerfidelity In-ear

In [39]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'innerfidelity', 'data', 'in-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Innerfidelity', 'in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'innerfidelity_autoeq_in-ear.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Innerfidelity Earbud

In [38]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'innerfidelity', 'data', 'earbud'),
    output_dir=ROOT_PATH.joinpath('results', 'Innerfidelity', 'earbud'),
    compensation=ROOT_PATH.joinpath('compensation', 'innerfidelity_autoeq_in-ear.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Headphone.com Legacy Over-ear

In [37]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'headphonecom', 'data', 'over-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Headphone.com Legacy', 'over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'headphonecom_harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Headphone.com Legacy In-ear

In [36]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'headphonecom', 'data', 'in-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Headphone.com Legacy', 'in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'headphonecom_autoeq_in-ear.csv'),
    bass_boost_gain=9.5, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Headphone.com Legacy Earbud

In [34]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'headphonecom', 'data', 'earbud'),
    output_dir=ROOT_PATH.joinpath('results', 'Headphone.com Legacy', 'earbud'),
    compensation=ROOT_PATH.joinpath('compensation', 'headphonecom_autoeq_in-ear.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=False, **eq_kwargs)

  0%|          | 0/10 [00:00<?, ?it/s]

## Update Indexes
Updates recommended results, full results, DB specific results, HeSuVi results and ranking table.

In [35]:
update_all_indexes()

Creating recommendations index...
Creating full index...
Creating source indices...
Creating HeSuVi ZIP archive...


  0%|          | 0/3662 [00:00<?, ?it/s]

Creating ranking index...


  0%|          | 0/3662 [00:00<?, ?it/s]

#### Data for webapp

In [51]:
write_entries_and_measurements()

  0%|          | 0/4841 [00:00<?, ?it/s]

## Deploy
1. Add files to Git, commit and push
2. Upload webapp data to server